In [5]:
import os

# Definir entorn de Django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "core.settings")


'core.settings'

In [1]:
import os
import django
from asgiref.sync import sync_to_async
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Set up Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "core.settings")
django.setup()

# Import models after Django setup
from app.models import ClusterAnalysis, UploadedDataset

# Create async function to clear tables
async def clear_tables():
    await sync_to_async(ClusterAnalysis.objects.all().delete)()
    await sync_to_async(UploadedDataset.objects.all().delete)()
    print("Successfully emptied ClusterAnalysis and UploadedDataset tables.")

# Run the async function in the existing event loop
import asyncio
loop = asyncio.get_event_loop()
loop.run_until_complete(clear_tables())

App package initialized
2025-07-05 12:36:06,428 - app - DEBUG - App package initialized
App package initialized


BASE_DIR: C:\Python Projects\Comunica AI Studio
Successfully emptied ClusterAnalysis and UploadedDataset tables.


In [ ]:
from django.conf import settings

# Accedir a les rutes configurades a settings.py
TEMPLATE_DIR = settings.TEMPLATE_DIR
IMAGES_DIR = settings.IMAGES_DIR
IMAGES_JSON = settings.IMAGES_JSON

print("Ruta plantilles:", TEMPLATE_DIR)
print("Ruta imatges:", IMAGES_DIR)
print("Fitxer imatges JSON:", IMAGES_JSON)

Ruta plantilles: C:\Python Projects\Agentic Journey\app\input\templates
Ruta imatges: C:\Python Projects\Agentic Journey\app\input\images
Fitxer imatges JSON: C:\Python Projects\Agentic Journey\app\input\images.json


In [ ]:
# generate_file_context.py

import os
import json
import groq
import re
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()


def prompt_AI(query):
    """
    Envia una consulta a l'API de Groq i retorna la resposta
    """
    try:
        # Obtenir clau API
        api_key = os.getenv('GROQ_API_KEY')
        if not api_key:
            raise ValueError("GROQ_API_KEY no trobada a les variables d'entorn")

        # Crear client
        client = groq.Groq(api_key=api_key)

        # Fer la consulta
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": query,
                }
            ],
            #model="llama-3.1-8b-instant",
            model="deepseek-r1-distill-llama-70b",
            temperature=0.7,
            max_tokens=2048,
        )

        return chat_completion.choices[0].message.content

    except Exception as e:
        print(f"Error en prompt_AI: {e}")
        return f"Error: {str(e)}"

EXCLUDED_FILES = {
    # Config i control de dependències
    "requirements.txt", "requirements-dev.txt", "Pipfile", "Pipfile.lock",
    "package-lock.json", "yarn.lock",

    # Fitxers innecessaris o temporals
    ".DS_Store", ".env", ".env.example", ".python-version", "__init__.py",

    # Arxius generats
    "db.sqlite3", "db.sqlite3-journal", "manage.py", "asgi.py", "wsgi.py", "database.db", ".csv","file_context.json",

    # Fitxers de test o configs
    "pytest.ini", "conftest.py", "tox.ini", "mypy.ini", "settings_local.py",
    ".flake8", ".pylintrc", ".editorconfig",

    # Fitxers de versions, cache o entorns
    ".gitignore", ".gitattributes",
}

EXCLUDED_FOLDERS = {
    # Entorns virtuals i gestió
    ".venv", "venv", "env", ".env", ".idea", ".vscode", ".git",

    # Compilats o temporals
    "__pycache__", "node_modules", "dist", "build", "staticfiles", "media", "collected_static",

    # Carpetes típiques Django
    "migrations", "core/migrations", "*/migrations", "templates", "locale",

    # Altres carpetes comunes en projectes Django
    "tests", "test_data", "fixtures", "docs", "logs",
}


def is_valid_file(path: str) -> bool:
    norm_path = path.replace("\\", "/")
    parts = norm_path.split("/")

    if any(folder in parts for folder in EXCLUDED_FOLDERS):
        return False

    if os.path.basename(path) in EXCLUDED_FILES:
        return False

    return os.path.isfile(path)



def read_files_and_describe(folder: str) -> list:
    file_descriptions = []

    for root, _, files in os.walk(folder):
        for file in files:
            full_path = os.path.join(root, file)
            relative_path = os.path.relpath(full_path, folder)

            if not is_valid_file(relative_path):
                continue

            try:
                with open(full_path, 'r', encoding='utf-8') as f:
                    content = f.read()
            except Exception as e:
                print(f"⚠️ Could not read {relative_path}: {e}")
                continue

            prompt = f"Give a concise description (1-3 sentences) of what this file does:\n\n{content[:5000]}"
            description = prompt_AI(prompt)  # Calls your AI description function

            file_descriptions.append({
                "file": relative_path,
                "description": description.strip(),
                "char_count": len(content)
            })

    return file_descriptions

def save_context_as_json(folder: str, output_path: str = "file_context.json"):
    result = read_files_and_describe(folder)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2)
    print(f"📄 Context saved to {output_path} with {len(result)} files.")

if __name__ == "__main__":
    save_context_as_json(".")


⚠️ Could not read app\users\eloi\user_database.db: 'utf-8' codec can't decode byte 0x9c in position 31: invalid start byte
⚠️ Could not read app\users\eloi_free\user_database.db: 'utf-8' codec can't decode byte 0xd0 in position 99: invalid continuation byte
Error en prompt_AI: no healthy upstream
⚠️ Could not read app\users\eloi_premium\user_database.db: 'utf-8' codec can't decode byte 0x85 in position 31: invalid start byte
Error en prompt_AI: no healthy upstream
Error en prompt_AI: no healthy upstream
📄 Context saved to file_context.json with 30 files.


In [40]:
# FITXERS PROJECTE SENSE IA

import os
import json

EXCLUDED_FILES = {
    # Fitxers innecessaris o temporals
    ".DS_Store", ".env", ".env.example", ".python-version", "__init__.py",

    # Arxius generats
    "db.sqlite3", "db.sqlite3-journal", "manage.py", "asgi.py", "wsgi.py", "database.db", ".csv", "file_context.json",

    # Fitxers de test o configs
    "pytest.ini", "conftest.py", "tox.ini", "mypy.ini", "settings_local.py",
    ".flake8", ".pylintrc", ".editorconfig", 

    # Fitxers de versions, cache o entorns
    ".gitignore", ".gitattributes",
}

EXCLUDED_FOLDERS = {
    # Entorns virtuals i gestió
    ".venv", "venv", "env", ".env", ".idea", ".vscode", ".git", 

    # Compilats o temporals
    "__pycache__", "node_modules", "dist", "build", "staticfiles", "media", "collected_static", 

    # Carpetes típiques Django
    "migrations", "core/migrations", "*/migrations", "locale", 

    # Altres carpetes comunes en projectes Django
    "tests", "test_data", "fixtures", "docs", "logs",
}

def is_valid_file(path: str) -> bool:
    norm_path = path.replace("\\", "/")
    parts = norm_path.split("/")

    if any(folder in parts for folder in EXCLUDED_FOLDERS):
        return False

    if os.path.basename(path) in EXCLUDED_FILES:
        return False

    return os.path.isfile(path)

def read_files_and_lengths(folder: str) -> list:
    file_info = []

    for root, _, files in os.walk(folder):
        for file in files:
            full_path = os.path.join(root, file)
            relative_path = os.path.relpath(full_path, folder)

            if not is_valid_file(relative_path):
                continue

            try:
                with open(full_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                file_info.append({
                    "file": relative_path,
                    "length": len(content)
                })
            except Exception as e:
                print(f"⚠️ Could not read {relative_path}: {e}")
                continue

    return file_info

def save_context_as_json(folder: str, output_path: str = "file_context.json"):
    result = read_files_and_lengths(folder)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2)
    print(f"📄 Context saved to {output_path} with {len(result)} files.")

if __name__ == "__main__":
    save_context_as_json(".")


⚠️ Could not read app\users\eloi\user_database.db: 'utf-8' codec can't decode byte 0x9c in position 31: invalid start byte
⚠️ Could not read app\users\eloi_free\user_database.db: 'utf-8' codec can't decode byte 0xd0 in position 99: invalid continuation byte
⚠️ Could not read app\users\eloi_premium\user_database.db: 'utf-8' codec can't decode byte 0x85 in position 31: invalid start byte
📄 Context saved to file_context.json with 55 files.


In [51]:
import json
import re

def load_context(context_path: str) -> list:
    """
    Carrega el context des d'un fitxer JSON i retorna una llista de fitxers amb les seves descripcions.
    """
    try:
        with open(context_path, 'r', encoding='utf-8') as f:
            full_context = json.load(f)
        # Retorna només els fitxers i les seves descripcions
        return [{"file": item["file"], "char_count": item.get("char_count", 0)} for item in full_context]
    except Exception as e:
        return {"error": f"Error loading context: {str(e)}"}


def extract_json_array(text: str) -> list:
    """
    Busca una llista en el text entre `[]` i la retorna com a llista.
    Si no troba cap llista, retorna un missatge d'error.
    """
    try:
        # Busquem llistes dins del text entre []
        matches = re.findall(r"\[.*?\]", text, re.DOTALL)
        if matches:
            # Carreguem la primera llista trobada
            return json.loads(matches[0])  
        else:
            return {"error": "No valid JSON array found", "raw": text}
    except json.JSONDecodeError as e:
        return {"error": f"Invalid JSON: {str(e)}", "raw": text}

def extract_json_object(text: str) -> dict:
    """
    Busca un objecte JSON dins del text entre `{}` i el retorna com a diccionari.
    """
    try:
        # Busquem objectes JSON dins del text entre {}
        matches = re.findall(r"\{.*?\}", text, re.DOTALL)
        if matches:
            # Carreguem el primer objecte JSON trobat
            return json.loads(matches[0])  
        else:
            return {"error": "No valid JSON object found", "raw": text}
    except json.JSONDecodeError as e:
        return {"error": f"Invalid JSON: {str(e)}", "raw": text}

def get_files_to_read(question: str) -> list:
    """
    Retorna una llista de fitxers que necessiten ser llegits segons la pregunta de l'usuari.
    Simplement extreu la llista en format JSON que es troba entre [].
    """
    context = load_context("file_context.json")
    prompt = (
        f"You are a code assistant. The user asked: \"{question}\"\n"
        "Please return a JSON array of file paths that need to be read, based on the question.\n"
        f"These are the existing files in the project: {context}\n"
        "Only return a valid JSON array of file paths, do not return any reasoning or other sentences, only the JSON."
    )
    response = prompt_AI(prompt)
    print(f"🤖 AI Response (files to read):\n{response}\n")

    # Extraiem el JSON de llista des del text
    files_to_read = extract_json_array(response)
    
    if isinstance(files_to_read, list):
        print(f"✅ Extracted file paths: {files_to_read}")
        return files_to_read
    else:
        print("❌ Error: Expected a JSON array of file paths.")
        return []

def summarize_changes(question: str) -> dict:
    """
    Dada la pregunta de l'usuari, genera una resposta amb les modificacions.
    Extreu l'objecte JSON del text entre `{}`.
    """
    prompt = (
        f"You are a code assistant. The user asked: \"{question}\"\n"
        "Please return a JSON object with the necessary changes.\n"
        "Only return a valid JSON object with the keys 'file' and 'code', do not return any reasoning or other sentences, only the JSON."
    )
    response = prompt_AI(prompt)
    print(f"🤖 AI Response (modifications):\n{response}\n")

    # Extraiem el JSON de l'objecte des del text
    change = extract_json_object(response)
    
    if isinstance(change, dict) and "file" in change and "code" in change:
        print(f"✅ Extracted changes: {json.dumps(change, indent=2)}")
        return change
    else:
        print("❌ Error: Expected a JSON object with 'file' and 'code'.")
        print("Response: ", change)  # Mostrar la resposta per depurar
        return {}

def generate_file_from_change(change: dict, base_folder: str = "."):
    """
    Genera o modifica un fitxer segons les modificacions definides en 'change'.
    """
    if not isinstance(change, dict) or "file" not in change or "code" not in change:
        print("❌ Invalid change format.")
        return
    
    file_path = os.path.join(base_folder, change["file"])
    
    dir_name = os.path.dirname(file_path)
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name, exist_ok=True)
            print(f"✅ Created directory: {dir_name}")
        except Exception as e:
            print(f"❌ Error creating directory {dir_name}: {e}")
            return
    
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(change["code"].strip())
        print(f"✅ File '{file_path}' has been created/modified.")
    except Exception as e:
        print(f"❌ Error writing file {file_path}: {e}")

if __name__ == "__main__":
    question = input("❓ What would you like to modify or add?\n> ")
    
    # Primer pas: obtenir els fitxers que cal llegir
    files_to_read = get_files_to_read(question)
    
    # Segon pas: resumir els canvis a realitzar
    change = summarize_changes(question)
    
    if change:
        print("\n🧾 Planned Modification:")
        print(json.dumps(change, indent=2))

        confirm = input("\n🔁 Apply this change? (yes/no)\n> ")
        if confirm.lower() == "yes":
            generate_file_from_change(change)
        else:
            print("❌ Operation cancelled.")


🤖 AI Response (files to read):
<think>
Alright, I need to figure out which files to modify so that sale.html has the same style as survey.html. First, I should look at where both these files are located. From the given files, sale.html is in app/input/templates, and survey.html is also in the same directory. So both are template files.

I know that HTML templates often extend a base template, which usually contains the common styling and structure. In this project, the base templates are base.html and base_authenticated.html, both located in app/templates. These files likely contain the common CSS links and overall layout that all pages use.

Looking at the files, I see that there's a static CSS file at app/static/css/styles.css. This is probably where the styles for both pages are defined. If survey.html has a certain style, it might be using specific classes or IDs in the CSS. To make sale.html look the same, I might need to ensure that it uses the same classes and that the CSS file 